In [6]:
import sys
sys.path.append("../src/")

In [8]:
%load_ext autoreload
%autoreload 2
import training
from training import ModelRunner

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
args = "--epochs 2 --emsize 768 --nhid 1536 --bptt 10 --batch_size 4 --dropout 0 --nlayers 2 --log-interval 1 --lr 1"
runner = ModelRunner(args.split())

In [19]:
runner.evaluate()

Loading batch 0
Start model
Finish model
0.125 47.042755126953125
Loading batch 1
Start model
Finish model
0.05 51.24909210205078
Loading batch 2
Start model
Finish model
0.05 43.89897918701172
Loading batch 3
Start model
Finish model
0.125 53.84113693237305
Loading batch 4
Start model
Finish model
0.05 77.6212387084961
Loading batch 5
Start model
Finish model
0.05 93.9007339477539
Loading batch 6
Start model
Finish model
0.1 97.24732971191406
Loading batch 7
Start model
Finish model
0.125 54.366554260253906


KeyboardInterrupt: 

In [ ]:
for i in range(20):
    runner.train(i, 5)

| epoch   0 |     1/28203 batches | lr 1.00000 | ms/batch 1490.51 | loss  2.57 | acc 0.85


# Current Issues
* optimize batch sizes (check gpu usage)
* choose correct batch size, dropout, bptt, num of negatives, model size
* doesn't train better than loss .2 / ac ~87.5%

## Notes:
* slowest part is data loading at 0.07 sec
* 16 minutes per epoch (data to 5k)
* ACC starts to grow around loss 2 or 3
* lr = 10 might be good
* --bptt 256 --batch_size 16 --nlayers 2 works --- 85% GPU util


In [54]:
import torch
import numpy as np


In [57]:
x = torch.tensor(np.arange(25).reshape((5,5)))
x

tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24]])

In [64]:
torch.gather(x, 0, torch.arange(5).view(-1,5))

tensor([[ 0,  6, 12, 18, 24]])

In [73]:
negs = torch.randint(0, 5, (5,2))
print(negs)
trues = torch.arange(5).view(5,-1)
print(trues)
both = torch.cat([trues, negs], 1)
print(both)

tensor([[1, 1],
        [0, 1],
        [3, 0],
        [0, 4],
        [4, 4]])
tensor([[0],
        [1],
        [2],
        [3],
        [4]])
tensor([[0, 1, 1],
        [1, 0, 1],
        [2, 3, 0],
        [3, 0, 4],
        [4, 4, 4]])


In [105]:
torch.gather(x, 1, both)

tensor([[ 0,  1,  1],
        [ 6,  5,  6],
        [12, 13, 10],
        [18, 15, 19],
        [24, 24, 24]])

In [104]:
total_size = 40
window_size = 4
np_negs = []
for i in range(total_size):
    choices = list(range(total_size))
    del choices[i]
    np_negs.append(np.random.choice(choices, window_size, replace=False))
np_negs = np.stack(np_negs)
print(np_negs)
np_trues = np.arange(total_size).reshape(total_size, 1)
all_options = np.concatenate([np_trues, np_negs], 1)

torch.tensor(all_options)

[[ 6 24 21 18]
 [10 34  0 36]
 [30 38 36  9]
 [10 36 25 26]
 [33 15 30 31]
 [23  2  7 25]
 [ 5 25 19  2]
 [ 2 28 12  1]
 [12  2 18 26]
 [22  2 12 17]
 [34 23 13 16]
 [33 37 16  6]
 [17  2  6 28]
 [ 0 35 34 16]
 [12 21 17 35]
 [23  5 31 34]
 [13 28  5  4]
 [34 32 23 37]
 [ 9 30 39  8]
 [37  6  1 38]
 [38 30 15 26]
 [ 9 32 10 38]
 [32  5 30 31]
 [ 3  7 16 18]
 [13  7 23 38]
 [ 8  9 17 15]
 [39 13 35 29]
 [25  4 28 17]
 [ 8 25 38 30]
 [26 25 12 32]
 [ 7 25 31 28]
 [ 0  8 23 14]
 [30 10 26 16]
 [26 25 22  0]
 [35  6  4 25]
 [26 28 38  3]
 [24  3 25 26]
 [26 27 10 33]
 [18 25  7  8]
 [ 0 27  7 11]]


tensor([[ 0,  6, 24, 21, 18],
        [ 1, 10, 34,  0, 36],
        [ 2, 30, 38, 36,  9],
        [ 3, 10, 36, 25, 26],
        [ 4, 33, 15, 30, 31],
        [ 5, 23,  2,  7, 25],
        [ 6,  5, 25, 19,  2],
        [ 7,  2, 28, 12,  1],
        [ 8, 12,  2, 18, 26],
        [ 9, 22,  2, 12, 17],
        [10, 34, 23, 13, 16],
        [11, 33, 37, 16,  6],
        [12, 17,  2,  6, 28],
        [13,  0, 35, 34, 16],
        [14, 12, 21, 17, 35],
        [15, 23,  5, 31, 34],
        [16, 13, 28,  5,  4],
        [17, 34, 32, 23, 37],
        [18,  9, 30, 39,  8],
        [19, 37,  6,  1, 38],
        [20, 38, 30, 15, 26],
        [21,  9, 32, 10, 38],
        [22, 32,  5, 30, 31],
        [23,  3,  7, 16, 18],
        [24, 13,  7, 23, 38],
        [25,  8,  9, 17, 15],
        [26, 39, 13, 35, 29],
        [27, 25,  4, 28, 17],
        [28,  8, 25, 38, 30],
        [29, 26, 25, 12, 32],
        [30,  7, 25, 31, 28],
        [31,  0,  8, 23, 14],
        [32, 30, 10, 26, 16],
        [3

In [107]:
x = torch.tensor(np.arange(40*40).reshape((40,40)))
torch.gather(x, 1, torch.tensor(all_options))

tensor([[   0,    6,   24,   21,   18],
        [  41,   50,   74,   40,   76],
        [  82,  110,  118,  116,   89],
        [ 123,  130,  156,  145,  146],
        [ 164,  193,  175,  190,  191],
        [ 205,  223,  202,  207,  225],
        [ 246,  245,  265,  259,  242],
        [ 287,  282,  308,  292,  281],
        [ 328,  332,  322,  338,  346],
        [ 369,  382,  362,  372,  377],
        [ 410,  434,  423,  413,  416],
        [ 451,  473,  477,  456,  446],
        [ 492,  497,  482,  486,  508],
        [ 533,  520,  555,  554,  536],
        [ 574,  572,  581,  577,  595],
        [ 615,  623,  605,  631,  634],
        [ 656,  653,  668,  645,  644],
        [ 697,  714,  712,  703,  717],
        [ 738,  729,  750,  759,  728],
        [ 779,  797,  766,  761,  798],
        [ 820,  838,  830,  815,  826],
        [ 861,  849,  872,  850,  878],
        [ 902,  912,  885,  910,  911],
        [ 943,  923,  927,  936,  938],
        [ 984,  973,  967,  983,  998],
